## 3. SQL Programming

In this section you are going to use a small database for a digital media store. You can access the database in two different ways, it is completely up to you which one you choose:

1. You can use the `isqlite3` extension and execute SQL in the notebook.
2. You can use any software that works with SQLite3 databases (the SQLite3 database is stored in `exam.db`), but only to execute SQL written by you. You are **NOT** allowed to use any wizards or other graphical user interface to prepare SQL statements.

If you choose the third option, **remember to copy and paste your solutions in the corresponding answer cells in this notebook!**

The database schema is shown below and the CREATE TABLE statements can be found in Appendix A.

![](database.svg)

In [ ]:
%load_ext isqlite3
%sql_open exam.db

### Assignment 3.1 (2 points)

Write an SQL query to return all customers from USA.

Sort the result set by the last name and then by the first name.

Columns in the result set: all columns in the relevant table.

In [ ]:
%%sql ANSWER CELL 3.1 (DO NOT REMOVE OR CHANGE THIS LINE)

select * from Customer where Country is 'USA' order by LastName, FirstName

### Assignment 3.2 (2 points)

Write an SQL query to return the number of how many artists have their name starting with `A`. (Only consider `A` at the beginning of the whole name, e.g. *Alice in Chains* is one of these artists, but *Pedro Luis & A Parede* is not.)

In [ ]:
%%sql ANSWER CELL 3.2 (DO NOT REMOVE OR CHANGE THIS LINE)

select count(*) as 'No. of Artists name starting with "A" ' 
from Artist 
where Name like 'A%'

### Assignment 3.3 (2 points)

Write an SQL query to return the title of all albums together with the artist's name.

Sort the result set by the artist's name first, then by the album's title.

Columns in the result set: artist's name, album's title.

In [ ]:
%%sql ANSWER CELL 3.3 (DO NOT REMOVE OR CHANGE THIS LINE)

select ar.Name as "Artist's Name", al.Title as "Album's Title" 
from Album as al join Artist as ar on al.ArtistId = ar.ArtistId

### Assignment 3.4 (2 points + possibility for 1 bonus point)

Write an SQL query to return the last name and the title of all employees, together with the last name and the title of their supervisor (i.e. the person they report to).

The list must include all employees, also those that do not have any supervisor.

Columns in the result set: employee's last name, employee's title, supervisor's last name, supervisor's first name.

#### Bonus point challenge

Add another column in the result set that shows employees' depth in the organizational hierarchy. Employees that do not have any supervisor are at level 1, employees reporting to them are at level 2, etc.

In [ ]:
%%sql ANSWER CELL 3.4 (DO NOT REMOVE OR CHANGE THIS LINE)

select e.Lastname as "Employee Last Name", e.Title as "Employee Title", s.Lastname as "Supervisor lastName",s.FirstName as "Supervisor Firstname"
from Employee as e left join Employee as s on e.ReportsTo = s.EmployeeId

### Assignment 3.5 (2 points)

Write an SQL query to return the ID, the last name and the title of all employees that do not support any customers.

Return the following columns in the result set: employee ID, last name, title.

In [ ]:
%%sql ANSWER CELL 3.5 (DO NOT REMOVE OR CHANGE THIS LINE)

select e.EmployeeId,e.LastName,e.Title from Employee as e left join Customer as c on c.SupportRepId = e.EmployeeId
where e.EmployeeId NOT in (select SupportRepId from Customer)

### Assignment 3.6 (2 points)

Write an SQL query to return the ID, the first and last name of all customers that have purchased at least one track in the store, together with the total amount they have spent (for all their purchases). 

Columns in the result set: customer ID, first name, last name, total amount.

In [ ]:
%%sql ANSWER CELL 3.6 (DO NOT REMOVE OR CHANGE THIS LINE)

select c.CustomerId as CusotmerID,c.FirstName as FirstName,c.LastName as LastName, sum(i.Total) as "Total Amount" 
from Customer as c join Invoice as i on c.CustomerId = i.CustomerId
group by i.CustomerId

### Assignment 3.7 (2 points)

Write an SQL query to select the ID and the name of playlists, together with their total duration in minutes.

Do not include playlists shorter than 120 minutes.

Sort the playlists by the duration in the descending order.

Columns in the result set: playlist ID, name, duration.

In [ ]:
%%sql ANSWER CELL 3.7 (DO NOT REMOVE OR CHANGE THIS LINE)

Select * from (Select pl.PlaylistId, pl.name, sum(t.Milliseconds)/60000 as duration
from Playlist as pl,PlaylistTrack as plt,Track as t
where pl.PlaylistId=plt.PlaylistId and plt.TrackId = t.TrackId
group by pl.PlaylistId) 
where duration>=120 
order by duration desc

### Assignment 3.8 (2 points)

As you might have noticed in the result set of the previous task, some of the playlists do not have unique names. Write an SQL query to select the ID and the title of all playlists that do not have unique names.

Columns to select: all columns in the playlist table.

In [ ]:
%%sql ANSWER CELL 3.8 (DO NOT REMOVE OR CHANGE THIS LINE)

Select * from (Select pl.PlaylistId, pl.name, sum(t.Milliseconds)/60000 as duration
from Playlist as pl,PlaylistTrack as plt,Track as t
where pl.PlaylistId=plt.PlaylistId and plt.TrackId = t.TrackId
group by pl.PlaylistId) 
where duration>=120 
order by duration desc

### Assignment 3.9 (2 points)

Write an SQL query to return the name of a city (or cities if there are several of them) where most of the employees live, together with the number of employees living there.

Columns in the result set: city, the number of employees living there.

In [ ]:
%%sql ANSWER CELL 3.9 (DO NOT REMOVE OR CHANGE THIS LINE)
select city, count(*) as Employees_Living
from employee 
group by city
having Employees_Living=
(select max(c) from(select count(*) as c from employee group by city))


### Assignment 3.10 (2 points)

a) Write an INSERT statement that fails due to the referential integrity.  
b) Using your example, explain what referential integrity means and why the statement failed.

In [ ]:
%%sql ANSWER CELL 3.10a (DO NOT REMOVE OR CHANGE THIS LINE)


insert into PlaylistTrack (PlaylistId,TrackId) values(40,4000)

### Used resources

Include URLs of online resources that you have used to solve the exercises in this section:

## Appendix A. Database schema

Note: `NVARCHAR`, `INTEGER` and `NUMERIC` are synonymous with `VARCHAR`, `INT` and `DECIMAL`.

```sql
CREATE TABLE Album
(
   AlbumId INTEGER NOT NULL,
   Title NVARCHAR(160) NOT NULL,
   ArtistId INTEGER NOT NULL,
   PRIMARY KEY (AlbumId),
   FOREIGN KEY (ArtistId) REFERENCES Artist (ArtistId)
);
CREATE TABLE Artist
(
   ArtistId INTEGER NOT NULL,
   Name NVARCHAR(120),
   PRIMARY KEY (ArtistId)
);
CREATE TABLE Customer
(
   CustomerId INTEGER NOT NULL,
   FirstName NVARCHAR(40) NOT NULL,
   LastName NVARCHAR(20) NOT NULL,
   Company NVARCHAR(80),
   Address NVARCHAR(70),
   City NVARCHAR(40),
   State NVARCHAR(40),
   Country NVARCHAR(40),
   PostalCode NVARCHAR(10),
   Phone NVARCHAR(24),
   Fax NVARCHAR(24),
   Email NVARCHAR(60) NOT NULL,
   SupportRepId INTEGER,
   PRIMARY KEY (CustomerId),
   FOREIGN KEY (SupportRepId) REFERENCES Employee (EmployeeId)
);
CREATE TABLE Employee
(
   EmployeeId INTEGER NOT NULL,
   LastName NVARCHAR(20) NOT NULL,
   FirstName NVARCHAR(20) NOT NULL,
   Title NVARCHAR(30),
   ReportsTo INTEGER,
   BirthDate DATETIME,
   HireDate DATETIME,
   Address NVARCHAR(70),
   City NVARCHAR(40),
   State NVARCHAR(40),
   Country NVARCHAR(40),
   PostalCode NVARCHAR(10),
   Phone NVARCHAR(24),
   Fax NVARCHAR(24),
   Email NVARCHAR(60),
   PRIMARY KEY (EmployeeId),
   FOREIGN KEY (ReportsTo) REFERENCES Employee (EmployeeId)
);
CREATE TABLE Genre
(
   GenreId INTEGER NOT NULL,
   Name NVARCHAR(120),
   PRIMARY KEY (GenreId)
);
5
CREATE TABLE Invoice
(
   InvoiceId INTEGER NOT NULL,
   CustomerId INTEGER NOT NULL,
   InvoiceDate DATETIME NOT NULL,
   BillingAddress NVARCHAR(70),
   BillingCity NVARCHAR(40),
   BillingState NVARCHAR(40),
   BillingCountry NVARCHAR(40),
   BillingPostalCode NVARCHAR(10),
   Total NUMERIC(10,2) NOT NULL,
   PRIMARY KEY (InvoiceId),
   FOREIGN KEY (CustomerId) REFERENCES Customer (CustomerId)
);
CREATE TABLE InvoiceLine
(
   InvoiceLineId INTEGER NOT NULL,
   InvoiceId INTEGER NOT NULL,
   TrackId INTEGER NOT NULL,
   UnitPrice NUMERIC(10,2) NOT NULL,
   Quantity INTEGER NOT NULL,
   PRIMARY KEY (InvoiceLineId),
   FOREIGN KEY (InvoiceId) REFERENCES Invoice (InvoiceId),
   FOREIGN KEY (TrackId) REFERENCES Track (TrackId)
);
CREATE TABLE MediaType
(
   MediaTypeId INTEGER NOT NULL,
   Name NVARCHAR(120),
   PRIMARY KEY (MediaTypeId)
);
CREATE TABLE Playlist
(
   PlaylistId INTEGER NOT NULL,
   Name NVARCHAR(120),
   PRIMARY KEY (PlaylistId)
);
CREATE TABLE PlaylistTrack
(
   PlaylistId INTEGER NOT NULL,
   TrackId INTEGER NOT NULL,
   PRIMARY KEY (PlaylistId, TrackId),
   FOREIGN KEY (PlaylistId) REFERENCES Playlist (PlaylistId),
   FOREIGN KEY (TrackId) REFERENCES Track (TrackId)
);
CREATE TABLE Track
(
   TrackId INTEGER NOT NULL,
   Name NVARCHAR(200) NOT NULL,
   AlbumId INTEGER,
   MediaTypeId INTEGER NOT NULL,
   GenreId INTEGER,
   Composer NVARCHAR(220),
   Milliseconds INTEGER NOT NULL,
   Bytes INTEGER,
   UnitPrice NUMERIC(10,2) NOT NULL,
   PRIMARY KEY (TrackId),
   FOREIGN KEY (AlbumId) REFERENCES Album (AlbumId),
   FOREIGN KEY (GenreId) REFERENCES Genre (GenreId),
   FOREIGN KEY (MediaTypeId) REFERENCES MediaType (MediaTypeId)
);
```